In [4]:
from geopy.distance import geodesic
import xarray as xr
import numpy as np
import dask
import pandas as pd
from pathlib import Path

In [2]:
dask.config.set({'scheduler.default': 'threads', 'pool': dask.threaded.get, 'num_workers': 4})

# Seasonal folder

In [5]:
path = Path('../data/copernicus/seasonal_vars/')
target_lat = 45.287865
target_lon = 18.802666

dataframes = {
    'CDD_C3S': [],
    'CSU_C3S': []
}

In [6]:
for file in path.glob('*.nc'):  # assuming they are all .nc files
    # Check if the filename starts with 'CDD_C3S'
    if file.name.startswith('CDD_C3S'):
        ds = xr.open_dataset(file)
        nearest_data = ds.sel(lat=target_lat, lon=target_lon, method='nearest')
        df = nearest_data.to_dataframe().reset_index(level='bounds', drop=True)
        dataframes['CDD_C3S'].append(df)
    # Check if the filename starts with 'CSU_C3S'
    elif file.name.startswith('CSU_C3S'):
        ds = xr.open_dataset(file)
        nearest_data = ds.sel(lat=target_lat, lon=target_lon, method='nearest')
        df = nearest_data.to_dataframe().reset_index(level='bounds', drop=True)
        dataframes['CSU_C3S'].append(df)

In [7]:
result_CDD = pd.concat(dataframes['CDD_C3S'])
result_CSU = pd.concat(dataframes['CSU_C3S'])
result_CDD = result_CDD.sort_index()
result_CSU = result_CSU.sort_index()

In [14]:
mean_CDD_1971_2020 = result_CDD.loc['1971':'2020'].mean()
mean_CDD_2021_2050 = result_CDD.loc['2021':'2050'].mean()

mean_CSU_1971_2020 = result_CSU.loc['1971':'2020'].mean()
mean_CSU_2021_2050 = result_CSU.loc['2021':'2050'].mean()

/tmp/ipykernel_1049946/2198688843.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  mean_CDD_1971_2020 = result_CDD.loc['1971':'2020'].mean()
/tmp/ipykernel_1049946/2198688843.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean_CDD_1971_2020 = result_CDD.loc['1971':'2020'].mean()
/tmp/ipykernel_1049946/2198688843.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  mean_CDD_2021_2050 = result_CDD.loc['2021':'2050'].mean()
/tmp/ipykernel_1049946/2198688843.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  mean_CDD_2021_2050 = result_CDD.loc['2021':'2050']

In [16]:
print("Mean CDD (1971-2020):", mean_CDD_1971_2020['CDD'])
print("Mean CDD (2021-2050):", mean_CDD_2021_2050['CDD'])
print("Mean CSU (1971-2020):", mean_CSU_1971_2020['CSU'])
print("Mean CSU (2021-2050):", mean_CSU_2021_2050['CSU'])

Mean CDD (1971-2020): 13.161616325378418
Mean CDD (2021-2050): 13.672268867492676
Mean CSU (1971-2020): 14.666666984558105
Mean CSU (2021-2050): 21.210084915161133


## Historical folder

In [18]:
path = Path('../data/copernicus/yearly_vars/')
target_lat = 45.287865
target_lon = 18.802666

dataframes = {
    'R20mm': [],
    'RR': [],
    'SU': [],
    'TG': [],
    'TR': [],
}
for file in path.glob('*.nc'):  # assuming they are all .nc files
    for prefix, dfs in dataframes.items():
        if file.name.startswith(prefix):
            ds = xr.open_dataset(file)
            nearest_data = ds.sel(lat=target_lat, lon=target_lon, method='nearest')
            df = nearest_data.to_dataframe().reset_index(level='bounds', drop=True)
            dfs.append(df)

In [19]:
results = {}
for var_name, dfs in dataframes.items():
    concatenated_df = pd.concat(dfs)
    concatenated_df = concatenated_df.sort_index()
    results[var_name] = concatenated_df

* Heavy precipitation

In [22]:
results['R20mm'].resample('Y').sum().loc['1971':'2020'].mean()

/tmp/ipykernel_1049946/3162342403.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['R20mm'].resample('Y').sum().loc['1971':'2020'].mean()


lat      3258.00
lon      1350.00
R20mm       3.24
dtype: float64

In [23]:
results['R20mm'].resample('Y').sum().loc['2021':'2050'].mean()

/tmp/ipykernel_1049946/1142590946.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['R20mm'].resample('Y').sum().loc['2021':'2050'].mean()


lat      3258.000000
lon      1350.000000
R20mm       3.333333
dtype: float64

* rainfall rate

In [25]:
results['RR'].resample('Y').mean().loc['1971':'2020'].mean()

/tmp/ipykernel_1049946/234440657.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['RR'].resample('Y').mean().loc['1971':'2020'].mean()


lat    45.250000
lon    18.750000
RR     21.344044
dtype: float64

In [26]:
results['RR'].resample('Y').mean().loc['2021':'2050'].mean()

/tmp/ipykernel_1049946/2869325977.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['RR'].resample('Y').mean().loc['2021':'2050'].mean()


lat    45.250000
lon    18.750000
RR     21.474613
dtype: float64

* summer days

In [34]:
results['SU'].reset_index('threshold')

,threshold,lat,lon,time_bounds,SU
time,,,,,
1951-01-05,20.0,45.25,18.75,1951-01-01,0.0
1951-01-05,20.0,45.25,18.75,1951-01-11,0.0
1951-01-05,25.0,45.25,18.75,1951-01-01,0.0
1951-01-05,25.0,45.25,18.75,1951-01-11,0.0
1951-01-05,30.0,45.25,18.75,1951-01-01,0.0
...,...,...,...,...,...
2070-12-25,25.0,45.25,18.75,2071-01-01,0.0
2070-12-25,30.0,45.25,18.75,2070-12-21,0.0
2070-12-25,30.0,45.25,18.75,2071-01-01,0.0


In [36]:
results['SU'].reset_index('threshold')[results['SU'].reset_index('threshold')['threshold'] == 30.0].resample('Y').sum().loc['1971':'2020'].mean()

/tmp/ipykernel_1049946/796382861.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['SU'].reset_index('threshold')[results['SU'].reset_index('threshold')['threshold'] == 30.0].resample('Y').sum().loc['1971':'2020'].mean()


threshold    2160.00
lat          3258.00
lon          1350.00
SU             47.68
dtype: float64

In [37]:
results['SU'].reset_index('threshold')[results['SU'].reset_index('threshold')['threshold'] == 30.0].resample('Y').sum().loc['2021':'2050'].mean()

/tmp/ipykernel_1049946/3939068461.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['SU'].reset_index('threshold')[results['SU'].reset_index('threshold')['threshold'] == 30.0].resample('Y').sum().loc['2021':'2050'].mean()


threshold    2160.000000
lat          3258.000000
lon          1350.000000
SU             83.666664
dtype: float64

* Mean daily temp

In [38]:
results['TG'].resample('Y').mean().loc['1971':'2020'].mean()

/tmp/ipykernel_1049946/1023986899.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['TG'].resample('Y').mean().loc['1971':'2020'].mean()


lat     45.250000
lon     18.750000
TG     285.158539
dtype: float64

In [39]:
results['TG'].resample('Y').mean().loc['2021':'2050'].mean()

/tmp/ipykernel_1049946/3695928439.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['TG'].resample('Y').mean().loc['2021':'2050'].mean()


lat     45.250000
lon     18.750000
TG     286.513702
dtype: float64

* Tropical nights

In [41]:
results['TR'].resample('Y').sum().loc['1971':'2020'].mean()

/tmp/ipykernel_1049946/993430843.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['TR'].resample('Y').sum().loc['1971':'2020'].mean()


lat    3258.00
lon    1350.00
TR       10.12
dtype: float64

In [44]:
results['TR'].resample('Y').sum().loc['2021':'2050'].mean()

/tmp/ipykernel_1049946/478428044.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results['TR'].resample('Y').sum().loc['2021':'2050'].mean()


lat    3258.0
lon    1350.0
TR       27.0
dtype: float64